In [1]:
from models import *

from util import Dictionary, get_args

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable

import json
import time
import random
import os

%load_ext autoreload
%autoreload 2

In [2]:
dropout = 0.5
n_token = 9326
nlayers = 2
nhid = 300
emsize = 300
attention_unit = 350
attention_hops = 4
nfc = 300
dictionary = Dictionary("/home/public/Yelp/small/dict_review_short.json")
word_vector = '/home/mlelarge/Structured-Self-Attentive-Sentence-Embedding/.vector_cache/glove.42B.300d.txt.pt'
class_number = 5

In [58]:
model = torch.load('/home/mlelarge/Structured-Self-Attentive-Sentence-Embedding/models/model-medium5.best_acc.pt')

In [62]:
data_val = open("/home/public/Yelp/small/val_tok.json").readlines()
dictionary = Dictionary(path="/home/public/Yelp/dict_review.json")

In [131]:
def package(data):
    """Package data for training / evaluation."""
    data = list(map(lambda x: json.loads(x), data))
    dat = list(map(lambda x: list(map(lambda y: dictionary.word2idx[y], x['text'])), data))
    maxlen = 0
    for item in dat:
        maxlen = max(maxlen, len(item))
    targets = list(map(lambda x: x['label'], data))
    maxlen = min(maxlen, 500)
    for i in range(len(data)):
        if maxlen < len(dat[i]):
            dat[i] = dat[i][:maxlen]
        else:
            for j in range(maxlen - len(dat[i])):
                dat[i].append(dictionary.word2idx['<pad>'])
    dat = torch.LongTensor(dat)
    targets = torch.LongTensor(targets)
    return dat.t(), targets

i=10
data, targets = package(data_val[i:i+10])
data = data.cuda()
targets = targets.cuda()

In [132]:
targets

tensor([4, 4, 4, 3, 2, 0, 4, 3, 4, 3], device='cuda:0')

In [133]:
hidden = model.init_hidden(data.size(1))
output, attention, _ = model.forward(data, hidden)

In [134]:
attention.shape

torch.Size([10, 4, 453])

In [135]:
torch.sum(attention,2)

tensor([[1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000],
        [1.0000, 1.0000, 1.0000, 1.0000]], device='cuda:0',
       grad_fn=<SumBackward2>)

In [140]:
j = 0
_, ind_att = torch.max(attention[j,:,:],1)

In [141]:
ind_att

tensor([ 0, 57,  1, 56], device='cuda:0')

In [142]:
data.shape

torch.Size([453, 10])

In [143]:
print(targets[j])
for i in ind_att:    
    print(dictionary.idx2word[data[i,j]])

tensor(4, device='cuda:0')
rob
.
was
question


In [144]:
attention[j,:,:]

tensor([[1.0000e+00, 0.0000e+00, 4.8076e-31,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [9.7073e-29, 9.9998e-01, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00]], device='cuda:0', grad_fn=<SliceBackward>)

In [3]:
vectors = torch.load(word_vector)

In [7]:
len(vectors)

4

In [8]:
vectors[0]

[',',
 'the',
 '.',
 'and',
 'to',
 'of',
 'a',
 'in',
 '"',
 'is',
 'for',
 ':',
 'i',
 ')',
 'that',
 '(',
 'you',
 'it',
 'on',
 '-',
 'with',
 "'s",
 'this',
 'by',
 'are',
 'at',
 'as',
 'be',
 'from',
 'have',
 'was',
 'or',
 'your',
 'not',
 '...',
 'we',
 '!',
 'but',
 '?',
 'all',
 'will',
 'an',
 'my',
 'can',
 'they',
 "n't",
 'do',
 'he',
 'more',
 'if',
 'one',
 'has',
 '|',
 'so',
 'about',
 'new',
 'what',
 'his',
 'there',
 'up',
 'out',
 ';',
 'their',
 'our',
 "'",
 'like',
 'when',
 '$',
 'just',
 'time',
 '&',
 'me',
 'which',
 'who',
 'no',
 'would',
 '/',
 '1',
 'some',
 'get',
 '[',
 ']',
 'also',
 'other',
 'how',
 'may',
 'had',
 'am',
 'been',
 '2',
 'her',
 'were',
 'them',
 'people',
 'she',
 'any',
 'now',
 'only',
 'pm',
 'first',
 'than',
 'good',
 '>',
 'into',
 'its',
 'these',
 'us',
 'see',
 'here',
 'make',
 'home',
 '3',
 'very',
 'over',
 'most',
 'then',
 'know',
 'said',
 'after',
 'well',
 'use',
 'two',
 '%',
 'did',
 'could',
 'day',
 'great',

In [9]:
vectors[1]

{',': 0,
 'the': 1,
 '.': 2,
 'and': 3,
 'to': 4,
 'of': 5,
 'a': 6,
 'in': 7,
 '"': 8,
 'is': 9,
 'for': 10,
 ':': 11,
 'i': 12,
 ')': 13,
 'that': 14,
 '(': 15,
 'you': 16,
 'it': 17,
 'on': 18,
 '-': 19,
 'with': 20,
 "'s": 21,
 'this': 22,
 'by': 23,
 'are': 24,
 'at': 25,
 'as': 26,
 'be': 27,
 'from': 28,
 'have': 29,
 'was': 30,
 'or': 31,
 'your': 32,
 'not': 33,
 '...': 34,
 'we': 35,
 '!': 36,
 'but': 37,
 '?': 38,
 'all': 39,
 'will': 40,
 'an': 41,
 'my': 42,
 'can': 43,
 'they': 44,
 "n't": 45,
 'do': 46,
 'he': 47,
 'more': 48,
 'if': 49,
 'one': 50,
 'has': 51,
 '|': 52,
 'so': 53,
 'about': 54,
 'new': 55,
 'what': 56,
 'his': 57,
 'there': 58,
 'up': 59,
 'out': 60,
 ';': 61,
 'their': 62,
 'our': 63,
 "'": 64,
 'like': 65,
 'when': 66,
 '$': 67,
 'just': 68,
 'time': 69,
 '&': 70,
 'me': 71,
 'which': 72,
 'who': 73,
 'no': 74,
 'would': 75,
 '/': 76,
 '1': 77,
 'some': 78,
 'get': 79,
 '[': 80,
 ']': 81,
 'also': 82,
 'other': 83,
 'how': 84,
 'may': 85,
 'had': 86,


In [10]:
vectors[2].shape

torch.Size([1917494, 300])

In [11]:
vectors[3]

300

In [39]:
model = Classifier({
        'dropout': dropout,
        'ntoken': n_token,
        'nlayers': nlayers,
        'nhid': nhid,
        'ninp': emsize,
        'pooling': 'all',
        'attention-unit': attention_unit,
        'attention-hops': attention_hops,
        'nfc': nfc,
        'dictionary': dictionary,
        'word-vector': word_vector,
        'class-number': class_number
    })

Loading word vectors from /home/mlelarge/Structured-Self-Attentive-Sentence-Embedding/.vector_cache/glove.42B.300d.txt.pt
300 300
9027 words from external word vectors loaded.


In [42]:
mat = torch.randn((10,4,4))

In [50]:
ret = (torch.sum(torch.sum((mat ** 2), 1), 1).squeeze() + 1e-10) ** 0.5

In [51]:
ret

tensor([3.8787, 3.1082, 3.6662, 4.3644, 3.1261, 3.4290, 4.6168, 3.9853, 3.4918,
        4.5313])

In [48]:
torch.sum((mat ** 2), 1)

torch.Size([10, 4])